In [20]:
import re
import tensorflow as tf
import tensorflow_datasets as tfds

In [11]:
# Tokenizer를 불러온다.
save_path = './CBot'
tokenizer = tfds.deprecated.text.SubwordTextEncoder.load_from_file(save_path + '/CBot_vocab')

In [16]:
import transformer

# Hyperparameter 정의
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1
MAX_LENGTH = 40
VOCAB_SIZE = tokenizer.vocab_size + 2

# 모델 정의 (학습한 모델과 같은 구조)
model = transformer.transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

# 이미 학습시켜놓은 가중치를 로드한다.
model.load_weights(save_path + '/CBot_weights')

In [17]:
def preprocess_sentence(sentence):
    """
    입력문장을 전처리하는 함수
    
    단어와 구두점 사이에 공백 추가.
    ex) 12시 땡! -> 12시 땡 !
    """
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

In [18]:
def evaluate(sentence):
    # 입력 문장 전처리
    sentence = preprocess_sentence(sentence)

    # 입력 문장에 시작 토큰과 종료 토큰을 추가
    sentence = tf.expand_dims(START_TKN + tokenizer.encode(sentence) + END_TKN, axis=0)
    output = tf.expand_dims(START_TKN, 0)

    # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

        # 현재 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if tf.equal(predicted_id, END_TKN[0]):
            break

        # 현재 시점의 예측 단어를 output(출력)에 연결한다.
        # output은 for문의 다음 루프에서 디코더의 입력이 된다.
        output = tf.concat([output, predicted_id], axis=-1)

    # 단어 예측이 모두 끝났다면 output을 리턴.
    return tf.squeeze(output, axis=0)

In [21]:
# import time
# t = time.time()
print('단순 채팅봇 CBot입니다.')
print('End, Exit 입력시 채팅봇이 종료됩니다.')
print()

while True:
    # Input
    sentence = input('User: ')
    
    # End, Exit => end the program
    end_words = ['END', 'EXIT', '종료', '끝']
    if sentence.upper() in end_words:
        break
    
    # Answer
    prediction = evaluate(sentence)
    answer = tokenizer.decode([i for i in prediction if i < tokenizer.vocab_size])
    answer = re.sub(r" ([?.!,])", r"\1", answer)    # 특수문자 전 후 공백 제거
    print(f"CBot: {answer}")

단순 채팅봇 CBot입니다.
End, Exit 입력시 채팅봇이 종료됩니다.

User: 안녕하세요


NameError: name 'START_TKN' is not defined